IMPORTS:

In [11]:
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
import csv
import time
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

ADDING LAT AND LONG VALUES:

In [13]:
df = pd.read_csv("avian-flu-global.csv")
df

,Year,Semester,World region,Country,Administrative Division,Disease,Serotype/Subtype/Genotype,Animal Category,Species,Event_id,Outbreak_id,New outbreaks,Susceptible,Measuring units,Cases,Killed and disposed of,Slaughtered,Deaths,Vaccinated
0,2009,Jul-Dec 2009,Americas,United States of America,Indiana,Influenza A virus (Inf. with),-,Domestic,Swine,753,12510,1,3000,Animal,-,0,0,0,-
1,2009,Jul-Dec 2009,Americas,United States of America,Virginia,Influenza A virus (Inf. with),-,Domestic,Birds,773,12648,1,3270,Animal,-,0,0,0,-
2,2009,Jul-Dec 2009,Asia,China (People's Rep. of),Beijing,Influenza A virus (Inf. with),-,Domestic,Dogs,765,12697,1,52,Animal,2,0,0,0,-
3,2009,Jul-Dec 2009,Asia,China (People's Rep. of),Heilongjiang,Influenza A virus (Inf. with),-,Domestic,Swine,765,12696,1,60,Animal,4,0,0,0,-
4,2009,Jul-Dec 2009,Asia,Thailand,Saraburi,Influenza A virus (Inf. with),-,Domestic,Swine,786,12715,1,680,Animal,34,0,0,0,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,2025,Jan-Jun 2025,Europe,Poland,Olszanica,Influenza A viruses of high pathogenicity (Inf...,H5N1,Domestic,Birds,4063,-,2,44,Animal,44,26,-,18,-
1480,2025,Jan-Jun 2025,Europe,Poland,Piecki,Influenza A viruses of high pathogenicity (Inf...,H5N1,Domestic,Birds,4063,148873,1,18,Animal,18,4,-,14,-
1481,2025,Jan-Jun 2025,Europe,Poland,Puławy,Influenza A viruses of high pathogenicity (Inf...,H5N1,Wild,Mute Swan,4063,148332,1,-,Animal,1,-,-,1,-
1482,2025,Jan-Jun 2025,Europe,Poland,Skępe,Influenza A viruses of high pathogenicity (Inf...,H5N1,Wild,Mute Swan,4063,148948,1,-,Animal,2,-,-,2,-


In [14]:
df['Country'].unique()

array(['United States of America', "China (People's Rep. of)", 'Thailand',
       'Australia', 'South Africa', 'Poland', 'Brazil',
       'Türkiye (Rep. of)'], dtype=object)

In [15]:
# change china to just china and get rid of parentheses

df.replace(to_replace ="China (People's Rep. of)",  value = "China",  inplace = True) 
df.replace(to_replace ="Türkiye (Rep. of)",  value = "Türkiye",  inplace = True) 
df.replace(to_replace ="Koszalin (City)",  value = "Koszalin",  inplace = True)
df.replace(to_replace ="Włocławek (City)",  value = "Włocławek",  inplace = True)
df.replace(to_replace ="Elbląg (City)",  value = "Elbląg",  inplace = True) 
df.replace(to_replace ="Grudziądz (City)",  value = "Grudziądz",  inplace = True)
df.replace(to_replace ="Konin (City)",  value = "Konin",  inplace = True)
df.replace(to_replace ="Opole (City)",  value = "Opole",  inplace = True) 
df.replace(to_replace ="Olsztyn (City)",  value = "Olsztyn",  inplace = True)
df.replace(to_replace ="Darłowo (rural)",  value = "Darłowo",  inplace = True)
df.replace(to_replace ="Iława (rural)",  value = "Iława",  inplace = True) 
df.replace(to_replace ="Puławy (rural)",  value = "Puławy",  inplace = True)
df.replace(to_replace ="Chojnów (rural)",  value = "Chojnów",  inplace = True)
df.replace(to_replace ="Złotów (rural)",  value = "Złotów",  inplace = True) 
df.replace(to_replace ="Kowal (rural)",  value = "Kowal",  inplace = True)
df.replace(to_replace ="Jarosław (rural)",  value = "Jarosław",  inplace = True)
df.replace(to_replace ="Lipno (rural)",  value = "Lipno",  inplace = True)
df.replace(to_replace ="Merkez",  value = "Merkez",  inplace = True)
df.replace(to_replace ="Lipno (rural)",  value = "Lipno",  inplace = True)



df.to_csv("adjusted.csv", index = False)

In [16]:
loc = Nominatim(user_agent="GetLoc")
with open('adjusted.csv', 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
      # To store modified rows
    header = next(reader)  # Extract header
    header.extend(["Latitude", "Longitude"])
    data = [header]

    for row in reader:
        if row[3] == row[4]:
            row.extend([None, None])
            data.append(row)
        else:
            location_query = f"{row[3]}, {row[4]}"  # Combine columns 3 and 4 for geocoding
            
            try:
                location = loc.geocode(location_query, timeout=10)
                if location:
                    row.extend([location.latitude, location.longitude])
                else:
                    row.extend([None, None])  # If location is not found
            except Exception as e:
                print(f"Error geocoding {location_query}: {e}")
                row.extend([None, None])  # Handle geocoding errors safely

            data.append(row)
            time.sleep(1)  # Pause to avoid hitting rate limits

In [17]:
output_file = "avian-flu-global-geocoded.csv"
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

print(f"Geocoding completed. Output saved to '{output_file}'.")

Geocoding completed. Output saved to 'avian-flu-global-geocoded.csv'.


In [18]:
df = pd.read_csv("avian-flu-global-geocoded.csv")
df

,Year,Semester,World region,Country,Administrative Division,Disease,Serotype/Subtype/Genotype,Animal Category,Species,Event_id,...,New outbreaks,Susceptible,Measuring units,Cases,Killed and disposed of,Slaughtered,Deaths,Vaccinated,Latitude,Longitude
0,2009,Jul-Dec 2009,Americas,United States of America,Indiana,Influenza A virus (Inf. with),-,Domestic,Swine,753,...,1,3000,Animal,-,0,0,0,-,40.327013,-86.174693
1,2009,Jul-Dec 2009,Americas,United States of America,Virginia,Influenza A virus (Inf. with),-,Domestic,Birds,773,...,1,3270,Animal,-,0,0,0,-,37.123224,-78.492772
2,2009,Jul-Dec 2009,Asia,China,Beijing,Influenza A virus (Inf. with),-,Domestic,Dogs,765,...,1,52,Animal,2,0,0,0,-,40.190632,116.412144
3,2009,Jul-Dec 2009,Asia,China,Heilongjiang,Influenza A virus (Inf. with),-,Domestic,Swine,765,...,1,60,Animal,4,0,0,0,-,48.000005,127.999992
4,2009,Jul-Dec 2009,Asia,Thailand,Saraburi,Influenza A virus (Inf. with),-,Domestic,Swine,786,...,1,680,Animal,34,0,0,0,-,14.624056,100.860102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,2025,Jan-Jun 2025,Europe,Poland,Olszanica,Influenza A viruses of high pathogenicity (Inf...,H5N1,Domestic,Birds,4063,...,2,44,Animal,44,26,-,18,-,52.860002,23.085551
1480,2025,Jan-Jun 2025,Europe,Poland,Piecki,Influenza A viruses of high pathogenicity (Inf...,H5N1,Domestic,Birds,4063,...,1,18,Animal,18,4,-,14,-,53.749179,21.341220
1481,2025,Jan-Jun 2025,Europe,Poland,Puławy,Influenza A viruses of high pathogenicity (Inf...,H5N1,Wild,Mute Swan,4063,...,1,-,Animal,1,-,-,1,-,51.426260,21.985301
1482,2025,Jan-Jun 2025,Europe,Poland,Skępe,Influenza A viruses of high pathogenicity (Inf...,H5N1,Wild,Mute Swan,4063,...,1,-,Animal,2,-,-,2,-,52.867156,19.346520


In [19]:
df.isna().sum()

Year                           0
Semester                       0
World region                   0
Country                        0
Administrative Division        0
Disease                        0
Serotype/Subtype/Genotype     35
Animal Category                0
Species                      248
Event_id                       0
Outbreak_id                    0
New outbreaks                  0
Susceptible                    0
Measuring units                0
Cases                          0
Killed and disposed of         0
Slaughtered                    0
Deaths                         0
Vaccinated                     0
Latitude                     472
Longitude                    472
dtype: int64

In [ ]:
df['Serotype/Subtype/Genotype'] = df['Serotype/Subtype/Genotype'].replace("-", "unknown")
df['Serotype/Subtype/Genotype'] = df['Serotype/Subtype/Genotype'].replace("not typed", "unknown")
df['Serotype/Subtype/Genotype'] = df['Serotype/Subtype/Genotype'].fillna("unknown")
df['Species'] =df['Species'].fillna("unknown")


In [21]:
df.isna().sum()

Year                           0
Semester                       0
World region                   0
Country                        0
Administrative Division        0
Disease                        0
Serotype/Subtype/Genotype      0
Animal Category                0
Species                        0
Event_id                       0
Outbreak_id                    0
New outbreaks                  0
Susceptible                    0
Measuring units                0
Cases                          0
Killed and disposed of         0
Slaughtered                    0
Deaths                         0
Vaccinated                     0
Latitude                     472
Longitude                    472
dtype: int64

In [22]:
unknown_df = df.loc[df.isnull().any(axis=1)]
unknown_df['Administrative Division']

43                              Zachodnio-Pomorskie
44                              Zachodnio-Pomorskie
45                              Zachodnio-Pomorskie
345                        United States of America
346                        United States of America
                           ...                     
1218                       United States of America
1219                       United States of America
1220                       United States of America
1221                       United States of America
1248    Unincorporated Australian Capital Territory
Name: Administrative Division, Length: 472, dtype: object

In [23]:
condition = df['Administrative Division'] == 'Zachodnio-Pomorskie'
df.loc[condition, 'Latitude'] = 53.4658
df.loc[condition, 'Longitude'] = 15.1823

In [24]:
condition = df['Administrative Division'] == 'Polish Exclusive Economic Zone'
df.loc[condition, 'Latitude'] = 54.9123
df.loc[condition, 'Longitude'] = 17.26042

In [25]:
condition = df['Administrative Division'] == 'Warszaw Zachodni'
df.loc[condition, 'Latitude'] = 52.2200
df.loc[condition, 'Longitude'] = 20.9653

In [26]:
condition = df['Administrative Division'] == 'Brazilian Exclusive Economic Zone'
df.loc[condition, 'Latitude'] = -11.09799
df.loc[condition, 'Longitude'] = -39.44423

In [27]:
condition = df['Administrative Division'] == 'Chinese Exclusive Economic Zone'
df.loc[condition, 'Latitude'] = 27.73853
df.loc[condition, 'Longitude'] = 118.1457

In [28]:
condition = df['Administrative Division'] == 'Unincorporated Australian Capital Territory'
df.loc[condition, 'Latitude'] = -35.473469
df.loc[condition, 'Longitude'] = 149.012375

In [29]:
condition = df['Administrative Division'] == 'Merkez'
df.loc[condition, 'Latitude'], df.loc[condition, 'Longitude']= 41.0581, 28.9106 

In [30]:
condition = df['Administrative Division'] == 'Sivrihisar'
df.loc[condition, 'Latitude'], df.loc[condition, 'Longitude']= 39.4760, 31.5370

In [31]:
df.isna().sum()

Year                           0
Semester                       0
World region                   0
Country                        0
Administrative Division        0
Disease                        0
Serotype/Subtype/Genotype      0
Animal Category                0
Species                        0
Event_id                       0
Outbreak_id                    0
New outbreaks                  0
Susceptible                    0
Measuring units                0
Cases                          0
Killed and disposed of         0
Slaughtered                    0
Deaths                         0
Vaccinated                     0
Latitude                     444
Longitude                    444
dtype: int64

In [32]:
condition = df['Administrative Division']== "United States of America"
df.loc[condition, 'Latitude'], df.loc[condition, 'Longitude']= 38.7946, -106.5348

In [33]:
df.to_csv('afg_geocoded.csv', index=False)

In [34]:
df = pd.read_csv("afg_geocoded.csv")
df

,Year,Semester,World region,Country,Administrative Division,Disease,Serotype/Subtype/Genotype,Animal Category,Species,Event_id,...,New outbreaks,Susceptible,Measuring units,Cases,Killed and disposed of,Slaughtered,Deaths,Vaccinated,Latitude,Longitude
0,2009,Jul-Dec 2009,Americas,United States of America,Indiana,Influenza A virus (Inf. with),-,Domestic,Swine,753,...,1,3000,Animal,-,0,0,0,-,40.327013,-86.174693
1,2009,Jul-Dec 2009,Americas,United States of America,Virginia,Influenza A virus (Inf. with),-,Domestic,Birds,773,...,1,3270,Animal,-,0,0,0,-,37.123224,-78.492772
2,2009,Jul-Dec 2009,Asia,China,Beijing,Influenza A virus (Inf. with),-,Domestic,Dogs,765,...,1,52,Animal,2,0,0,0,-,40.190632,116.412144
3,2009,Jul-Dec 2009,Asia,China,Heilongjiang,Influenza A virus (Inf. with),-,Domestic,Swine,765,...,1,60,Animal,4,0,0,0,-,48.000005,127.999992
4,2009,Jul-Dec 2009,Asia,Thailand,Saraburi,Influenza A virus (Inf. with),-,Domestic,Swine,786,...,1,680,Animal,34,0,0,0,-,14.624056,100.860102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,2025,Jan-Jun 2025,Europe,Poland,Olszanica,Influenza A viruses of high pathogenicity (Inf...,H5N1,Domestic,Birds,4063,...,2,44,Animal,44,26,-,18,-,52.860002,23.085551
1480,2025,Jan-Jun 2025,Europe,Poland,Piecki,Influenza A viruses of high pathogenicity (Inf...,H5N1,Domestic,Birds,4063,...,1,18,Animal,18,4,-,14,-,53.749179,21.341220
1481,2025,Jan-Jun 2025,Europe,Poland,Puławy,Influenza A viruses of high pathogenicity (Inf...,H5N1,Wild,Mute Swan,4063,...,1,-,Animal,1,-,-,1,-,51.426260,21.985301
1482,2025,Jan-Jun 2025,Europe,Poland,Skępe,Influenza A viruses of high pathogenicity (Inf...,H5N1,Wild,Mute Swan,4063,...,1,-,Animal,2,-,-,2,-,52.867156,19.346520


In [35]:
df.isna().sum()

Year                         0
Semester                     0
World region                 0
Country                      0
Administrative Division      0
Disease                      0
Serotype/Subtype/Genotype    0
Animal Category              0
Species                      0
Event_id                     0
Outbreak_id                  0
New outbreaks                0
Susceptible                  0
Measuring units              0
Cases                        0
Killed and disposed of       0
Slaughtered                  0
Deaths                       0
Vaccinated                   0
Latitude                     0
Longitude                    0
dtype: int64

In [36]:
df = df.drop(columns= ['Outbreak_id', 'Slaughtered', 'Vaccinated'])
df

,Year,Semester,World region,Country,Administrative Division,Disease,Serotype/Subtype/Genotype,Animal Category,Species,Event_id,New outbreaks,Susceptible,Measuring units,Cases,Killed and disposed of,Deaths,Latitude,Longitude
0,2009,Jul-Dec 2009,Americas,United States of America,Indiana,Influenza A virus (Inf. with),-,Domestic,Swine,753,1,3000,Animal,-,0,0,40.327013,-86.174693
1,2009,Jul-Dec 2009,Americas,United States of America,Virginia,Influenza A virus (Inf. with),-,Domestic,Birds,773,1,3270,Animal,-,0,0,37.123224,-78.492772
2,2009,Jul-Dec 2009,Asia,China,Beijing,Influenza A virus (Inf. with),-,Domestic,Dogs,765,1,52,Animal,2,0,0,40.190632,116.412144
3,2009,Jul-Dec 2009,Asia,China,Heilongjiang,Influenza A virus (Inf. with),-,Domestic,Swine,765,1,60,Animal,4,0,0,48.000005,127.999992
4,2009,Jul-Dec 2009,Asia,Thailand,Saraburi,Influenza A virus (Inf. with),-,Domestic,Swine,786,1,680,Animal,34,0,0,14.624056,100.860102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,2025,Jan-Jun 2025,Europe,Poland,Olszanica,Influenza A viruses of high pathogenicity (Inf...,H5N1,Domestic,Birds,4063,2,44,Animal,44,26,18,52.860002,23.085551
1480,2025,Jan-Jun 2025,Europe,Poland,Piecki,Influenza A viruses of high pathogenicity (Inf...,H5N1,Domestic,Birds,4063,1,18,Animal,18,4,14,53.749179,21.341220
1481,2025,Jan-Jun 2025,Europe,Poland,Puławy,Influenza A viruses of high pathogenicity (Inf...,H5N1,Wild,Mute Swan,4063,1,-,Animal,1,-,1,51.426260,21.985301
1482,2025,Jan-Jun 2025,Europe,Poland,Skępe,Influenza A viruses of high pathogenicity (Inf...,H5N1,Wild,Mute Swan,4063,1,-,Animal,2,-,2,52.867156,19.346520


In [37]:
df.isna().sum()

Year                         0
Semester                     0
World region                 0
Country                      0
Administrative Division      0
Disease                      0
Serotype/Subtype/Genotype    0
Animal Category              0
Species                      0
Event_id                     0
New outbreaks                0
Susceptible                  0
Measuring units              0
Cases                        0
Killed and disposed of       0
Deaths                       0
Latitude                     0
Longitude                    0
dtype: int64

In [38]:
cols = ['New outbreaks', 'Susceptible', 'Cases', 'Killed and disposed of', 'Deaths']
for col in cols:
    df[col] = df[col].replace('-', np.nan)

In [39]:
numeric_features = ['New outbreaks', 'Susceptible', 'Cases', 'Killed and disposed of', 'Deaths', 'Latitude', 'Longitude', 'Year']
categorical_features = [col for col in df.columns if col not in numeric_features] 
imputer = KNNImputer(n_neighbors=5)
df_numeric_imputed = pd.DataFrame(imputer.fit_transform(df[numeric_features]), columns=numeric_features)
df_imputed = pd.concat([df[categorical_features].reset_index(drop=True), df_numeric_imputed], axis=1)
df = df_imputed
df.isna().sum()
df.to_csv('afg_geocoded_imputed.csv', index=False)

,Semester,World region,Country,Administrative Division,Disease,Serotype/Subtype/Genotype,Animal Category,Species,Measuring units,New outbreaks,Susceptible,Cases,Killed and disposed of,Deaths,Latitude,Longitude,Year
0,Jul-Dec 2009,Americas,United States of America,Indiana,Influenza A virus (Inf. with),-,Domestic,Swine,Animal,1.0,3000.0,1.0,0.0,0.0,40.327013,-86.174693,2009.0
1,Jul-Dec 2009,Americas,United States of America,Virginia,Influenza A virus (Inf. with),-,Domestic,Birds,Animal,1.0,3270.0,1.2,0.0,0.0,37.123224,-78.492772,2009.0
2,Jul-Dec 2009,Asia,China,Beijing,Influenza A virus (Inf. with),-,Domestic,Dogs,Animal,1.0,52.0,2.0,0.0,0.0,40.190632,116.412144,2009.0
3,Jul-Dec 2009,Asia,China,Heilongjiang,Influenza A virus (Inf. with),-,Domestic,Swine,Animal,1.0,60.0,4.0,0.0,0.0,48.000005,127.999992,2009.0
4,Jul-Dec 2009,Asia,Thailand,Saraburi,Influenza A virus (Inf. with),-,Domestic,Swine,Animal,1.0,680.0,34.0,0.0,0.0,14.624056,100.860102,2009.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,Jan-Jun 2025,Europe,Poland,Olszanica,Influenza A viruses of high pathogenicity (Inf...,H5N1,Domestic,Birds,Animal,2.0,44.0,44.0,26.0,18.0,52.860002,23.085551,2025.0
1480,Jan-Jun 2025,Europe,Poland,Piecki,Influenza A viruses of high pathogenicity (Inf...,H5N1,Domestic,Birds,Animal,1.0,18.0,18.0,4.0,14.0,53.749179,21.341220,2025.0
1481,Jan-Jun 2025,Europe,Poland,Puławy,Influenza A viruses of high pathogenicity (Inf...,H5N1,Wild,Mute Swan,Animal,1.0,3.0,1.0,0.2,1.0,51.426260,21.985301,2025.0
1482,Jan-Jun 2025,Europe,Poland,Skępe,Influenza A viruses of high pathogenicity (Inf...,H5N1,Wild,Mute Swan,Animal,1.0,4.8,2.0,0.2,2.0,52.867156,19.346520,2025.0
